In [ ]:
import pandas as pd
import numpy as np

In [ ]:
d0 = pd.read_excel('/home/eduardo/repositorios/IS_Agro/databases/bronze_mapbiomas_brasil_col9_state_municipality.xlsx', sheet_name=1)
d0['geocode'] = d0['geocode'].astype(str)

## ChatGPT

In [ ]:
# Anos
seqano = [year for year in range(1985, 2023+1)]

# Sigla
d = d0.copy()
d['state_acronym'] = d0['municipality - state'].str[-2:]

# Seleção de colunas
d = d[['geocode', 'municipality', 'municipality - state', 'state', 'class', 'state_acronym'] + seqano]

In [ ]:
# Subconjunto para classe mosaico
dm1 = d[d['class'] == 21].copy()
dm1.set_index('geocode', inplace=True)

# Subconjunto para pastagem, temporárias e perenes
dm2 = d[d['class'].isin([15, 41, 48]) & d['geocode'].isin(dm1.index)].copy()

In [ ]:
# Consolidando área total por município
dm2s = dm2.groupby('geocode')[seqano].sum()

frac1 = dm2[seqano].div(dm2s.loc[dm2['geocode']].values)

In [ ]:
# Consolidando por estado
dm3 = d.groupby(['state', 'state_acronym', 'class'])[seqano].sum().reset_index()
dm3.set_index(dm3['state_acronym'] + '-' + dm3['class'].astype(str), inplace=True)

dm4 = dm3.loc[dm2['state_acronym'] + '-' + dm2['class'].astype(str)].copy()
dm4['geocode'] = dm2['geocode'].values

dm4s = dm4.groupby('geocode')[seqano].sum()

frac2 = dm4[seqano].div(dm4s.loc[dm4['geocode']].values)

In [ ]:
# Reclassificação para mosaico
frac = pd.DataFrame(np.where(dm2s.loc[dm2['geocode']].values != 0, frac1, frac2))

dmtpp = dm2.copy()
dmtpp[seqano] = dm1.loc[dm2['geocode'], seqano].values * frac.values
dmtpp['class_level_2'] = 'Land Use Mosaic'

In [ ]:
# IDs de municípios faltantes
idsgeo = set(dm1.index) - set(dmtpp['geocode'])
idsuf = dm1.loc[dm1.index.isin(idsgeo), 'state_acronym'].unique()
idsmun = dm1.loc[dm1.index.isin(idsgeo), 'municipality'].unique()
idsmuf = dm1.loc[dm1.index.isin(idsgeo), 'municipality - state'].unique()

# Consolidação por estados faltantes
dm5 = dm3[(dm3['state_acronym'].isin(idsuf)) & (dm3['class'].isin([15, 41, 48]))].convert_dtypes()
dm5s = dm5.groupby(['state', 'state_acronym'])[seqano].sum()
dm5s = dm5s.reset_index().set_index('state_acronym')

dm5[seqano] = dm5[seqano].div(dm5s.loc[dm5['state_acronym']][seqano].values)
idrs = [dm5.index.get_loc(x) for x in dm5.index if any(uf in x for uf in idsuf)]

# Removendo ou tratando valores nulos antes de continuar
idsgeo = [x for x in idsgeo if pd.notna(x)]
idsmuf = [x for x in idsmuf if pd.notna(x)]
idsmun = [x for x in idsmun if pd.notna(x)]

dm6 = pd.concat([pd.DataFrame({
    'geocode': list(idsgeo),  # Garantindo que é uma lista sem NaNs
    'municipality - state': list(idsmuf),  # Garantindo que é uma lista sem NaNs
    'municipality': list(idsmun)  # Garantindo que é uma lista sem NaNs
}),
dm5.iloc[idrs]], axis=1)

# Remova as linhas onde 'geocode' é NaN
print(dm6['geocode'].isna().sum())  # Conta a quantidade de NaNs na coluna geocode
dm6_clean = dm6.dropna(subset=['geocode'])
dm6_clean[seqano] = dm6_clean[seqano] * dm1.loc[dm6_clean['geocode'], seqano].values

# Definir 'class_level_2'
dm6_clean['class_level_2'] = 'Land Use Mosaic'

In [ ]:
dm3[(dm3['state_acronym'].isin(idsuf)) & (dm3['class'].isin([15, 41, 48]))]

## Claude

In [ ]:
# Anos
seqano = [year for year in range(1985, 2024)]

# Sigla
d = d0.copy()
d['state_acronym'] = d0['municipality - state'].str[-2:]

d = d[['geocode', 'municipality - state', 'municipality', 'state', 'state_acronym'] + list(d.columns[6:51])]

In [ ]:
# Subconjunto para classe mosaico
dm1 = d[d['class'] == 21]  # id 3.4. Mosaic of Uses
dm1 = dm1.set_index('geocode')

# Subconjunto para pastagem, temporárias e perenes
dm2 = d[(d['class'].isin([15, 41, 48])) & (d['geocode'].isin(dm1.index))]
dm2s = dm2.groupby('geocode')[seqano].sum().reset_index()
dm2s = dm2s.set_index('geocode')

frac1 = dm2[seqano].values / dm2s.loc[dm2['geocode'], seqano].values
print(f"NaN values in frac1: {np.isnan(frac1).sum()}")

In [ ]:
 # Consolidando por estado
dm3 = d.groupby(['state', 'state_acronym', 'class', 'class_level_0', 'class_level_1', 'class_level_2', 'class_level_3', 'class_level_4'])[seqano].sum().reset_index()
dm3.index = dm3['state_acronym'] + '-' + dm3['class'].astype(str)

dm4 = dm3.loc[dm2['state_acronym'] + '-' + dm2['class'].astype(str)]
dm4['geocode'] = dm2['geocode'].values
dm4s = dm4.groupby('geocode')[seqano].sum().reset_index()
dm4s = dm4s.set_index('geocode')

frac2 = dm4[seqano].values / dm4s.loc[dm4['geocode'], seqano].values
print(f"NaN values in frac2: {np.isnan(frac2).sum()}")
print(f"Zero values in dm4s: {(dm4s == 0).sum().sum()}")

In [ ]:
dm4

In [ ]:
frac = np.where(dm2s.loc[dm2['geocode'], seqano].values != 0, frac1, frac2)
print(f"NaN values in frac: {np.isnan(frac).sum()}")

dmtpp = dm2.copy()
dmtpp[seqano] = dm1.loc[dm2['geocode'], seqano].values * frac
dmtpp['class_level_2'] = 'Land Use Mosaic'

In [ ]:
idsgeo = list(set(dm1.index) - set(dmtpp['geocode']))
idsuf = dm1.loc[dm1.index.isin(idsgeo), 'state_acronym'].unique()
idsmun = dm1.loc[dm1.index.isin(idsgeo), 'municipality'].unique()
idsmuf = dm1.loc[dm1.index.isin(idsgeo), 'municipality - state'].unique()

print(f"Sum equality check: {np.allclose(dm1.loc[~dm1.index.isin(idsgeo), seqano].sum(), dmtpp[seqano].sum())}")

In [ ]:
dm5 = dm3[(dm3['state_acronym'].isin(idsuf)) & (dm3['class'].isin([15, 41, 48]))]
dm5s = dm5.groupby(['state', 'state_acronym'])[seqano].sum().reset_index()
dm5s = dm5s.set_index('state_acronym')

dm5[seqano] = dm5[seqano].values / dm5s.loc[dm5['state_acronym'], seqano].values

idrs = [i for uf in idsuf for i in dm5.index[dm5['state_acronym'] == uf]]

In [ ]:
dm6 = pd.DataFrame({
    'geocode': np.repeat(idsgeo, 3),
    'municipality - state': np.repeat(idsmuf, 3),
    'municipality': np.repeat(idsmun, 3)
})
dm6 = pd.concat([dm6, dm5.loc[idrs].reset_index(drop=True)], axis=1)
dm6[seqano] = dm6[seqano].values * dm1.loc[dm6['geocode'], seqano].values
dm6['class_level_2'] = 'Land Use Mosaic'

print(f"Sum difference check: {np.allclose(dm6[seqano].sum().round(6), dm1.loc[dm1['municipality'] == 'São Caetano do Sul', seqano].sum().round(6))}")

In [ ]:
dmtpp = pd.concat([dmtpp, dm6])

print(f"Final sum equality check: {np.allclose(dm1[seqano].sum(), dmtpp[seqano].sum())}")

In [ ]:
ds = pd.concat([d[d['class'] != 21], dmtpp])

In [ ]:
# VERIFICAÇÕES
print(f"Sum equality checks:")
print(f"d vs ds: {np.allclose(d[seqano].sum(), ds[seqano].sum())}")
print(f"d vs d0: {np.allclose(d[seqano].sum().round(6), d0[seqano].sum().round(6))}")
print(f"d0 vs ds: {np.allclose(d0[seqano].sum(), ds[seqano].sum())}")

print(f"Subset sum equality checks:")
print(f"d vs ds (excluding idsgeo): {np.allclose(d[~d['geocode'].isin(idsgeo)][seqano].sum(), ds[~ds['geocode'].isin(idsgeo)][seqano].sum())}")
print(f"d0 vs ds (excluding idsgeo): {np.allclose(d0[~d0['geocode'].isin(idsgeo)][seqano].sum(), ds[~ds['geocode'].isin(idsgeo)][seqano].sum())}")
print(f"d vs ds (all): {np.allclose(d[seqano].sum(), ds[seqano].sum())}")
print(f"d0 vs d (all): {np.allclose(d0[seqano].sum(), d[seqano].sum())}")
print(f"d0 vs ds (all, rounded): {np.allclose(d0[seqano].sum().round(6), ds[seqano].sum().round(6))}")

In [ ]:
tbr1 = ds.groupby(['class', 'class_level_0', 'class_level_1', 'class_level_2', 'class_level_3', 'class_level_4'])[seqano].sum().reset_index()
tbr2 = ds.groupby(['class_level_0', 'class_level_1', 'class_level_3'])[seqano].sum().reset_index()

ds2 = ds.groupby(['geocode', 'municipality - state', 'municipality', 'state', 'state_acronym', 'class_level_0', 'class_level_1', 'class_level_3'])[seqano].sum().reset_index()

In [ ]:
ds.reset_index(drop=True)

In [ ]:
ds2